In [25]:
#!pip install langchain
from langchain.text_splitter import CharacterTextSplitter
!python --version

Python 3.8.10


In [26]:
# import openai
# openai.Model.list()

Read OPENAI API KEY

In [27]:
from langchain.chat_models import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Number, extract_from_documents

with open("API_KEY", "r", encoding="utf-8") as f:
    key = f.read()

Generate LengChain
Instantiate a langchain LLM and create a chain.

In [28]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=900,
    openai_api_key=key
)

create schema for some category of pdf file

In [29]:
appeal = Object(
    id="appeal",
    description="上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等",
    attributes=[
        Text(id="agent", description="代理人", many=True),
        Text(id="appellant", description="上訴人", examples=[("原告　蔡宗甫\n", "蔡宗甫"),
                                                            ("聲請人　陳鴻斌\n", "陳鴻斌"),
                                                            ("聲請人　新鑫股份有限公司\n", "新鑫股份有限公司")], many=True),
        Text(id="representative", description="代表人", many=True),
        Text(id="attorney", description="訴訟代理人", examples=[("訴訟代理人　林虹妤\n", "林虹妤")], many=True),
        Text(id="reagent", description="複代理人", many=True),
        Text(id="legal_representative", description="法定代理人", examples=[("法定代理人　許國興\n", "許國興")], many=True),
        Text(id="defender", description="辯護人", many=True),
        Text(id="assistant", description="輔佐人", many=True)
    ]
)

appealed = Object(
    id="appealed",
    description="被上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等",
    attributes=[
        Text(id="agent", description="代理人", many=True),
        Text(id="appellee", description="被上訴人", examples=[("被告　臺北市交通事件裁決所\n", "臺北市交通事件裁決所"),
                                                             ("相對人　林佳璇\n", "林佳璇")], many=True),
        Text(id="representative", description="代表人", examples=[("代表人　蘇福智\n", "蘇福智")], many=True),
        Text(id="attorney", description="訴訟代理人", examples=[("訴訟代理人　黃郁軒\n", "黃郁軒")], many=True),
        Text(id="reagent", description="複代理人", many=True),
        Text(id="legal_representative", description="法定代理人", many=True),
        Text(id="defender", description="辯護人", many=True),
        Text(id="assistant", description="輔佐人", many=True)
    ]
)

schema_part1_1 = Object(
    id="judgment_info_part1",
    description="擷取分析裁判書的上半部分屬性值",
    attributes=[
        Text(
            id="title",
            description="這份裁判書的標題",
            examples=[("臺灣士林地方法院行政訴訟判決\n", "臺灣士林地方法院行政訴訟判決"),
                      ("司法院職務法庭裁定\n", "司法院職務法庭裁定"),
                      ("臺灣嘉義地方法院民事裁定\n", "臺灣嘉義地方法院民事裁定")],
        ),
        Text(
            id="id",
            description="這份裁判書的編號",
            examples=[("111年度交字第35號\n", "111年度交字第35號"),
                      ("108年度懲再字第5號\n", "108年度懲再字第5號"),
                      ("111年度司票字第434號\n", "111年度司票字第434號")],
        ),
        appeal,
        appealed,
        Text(
            id="foreword",
            description="這份裁判書的前言",
            examples=[("上列當事人間交通裁決事件，原告不服被告中華民國111年1月11\n日北市裁催字第22-AC0000000號、第22-AC0000000號裁決，提起\n行政訴訟，本院判決如下：\n", "上列當事人間交通裁決事件，原告不服被告中華民國111年1月11日北市裁催字第22-AC0000000號、第22-AC0000000號裁決，提起行政訴訟，本院判決如下："),
                      ("上列聲請人因懲戒事件，對於本庭中華民國108年8月27日108年\n度懲再字第1號裁定，聲請再審，本庭裁定如下：", "上列聲請人因懲戒事件，對於本庭中華民國108年8月27日108年度懲再字第1號裁定，聲請再審，本庭裁定如下："),
                      ("上列當事人間聲請對本票准許強制執行事件，本院裁定如下：\n", "上列當事人間聲請對本票准許強制執行事件，本院裁定如下：")],
        ),
        Text(
            id="main_content",
            description="這份裁判書的主文",
            examples=[("主文\n原告之訴駁回。\n訴訟費用新臺幣參佰元由原告負擔。\n", "原告之訴駁回。訴訟費用新臺幣參佰元由原告負擔。"),
                      ("主文\n再審之聲請駁回。\n", "再審之聲請駁回。"),
                      ("主文\n相對人簽發如附表所示之本票金額及自附表所載利息起算日起至\n清償日止按週年利率16%計算之利息，准予強制執行。\n聲請程序費用新臺幣2,000元由相對人負擔。","相對人簽發如附表所示之本票金額及自附表所載利息起算日起至清償日止按週年利率16%計算之利息，准予強制執行。聲請程序費用新臺幣2,000元由相對人負擔。")],
        )
    ],
    many=True
)

In [30]:
chain_part1 = create_extraction_chain(llm, schema_part1_1, encoder_or_encoder_class="JSON", input_formatter=None)
print(chain_part1.prompt.format_prompt("[user input]").to_string())


Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

judgment_info_part1: Array<{ // 擷取分析裁判書的上半部分屬性值
 title: string // 這份裁判書的標題
 id: string // 這份裁判書的編號
 appeal: { // 上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等
  agent: Array<string> // 代理人
  appellant: Array<string> // 上訴人
  representative: Array<string> // 代表人
  attorney: Array<string> // 訴訟代理人
  reagent: Array<string> // 複代理人
  legal_representative: Array<string> // 法定代理人
  defender: Array<string> // 辯護人
  assistant: Array<string> // 輔佐人
 }
 appealed: { // 被上訴人相關資訊;例如:法定代理人,訴訟代理人,代表人...等等
  agent: Array<string> // 代理人
  appellee: Array<string> // 被上訴人
  representative: Array<string> // 代表人
  attorney: Array<string> // 訴訟代理人
  reagent: Array<string> // 複代理人
  legal_representative: Array<string> // 法定代理人
  defender: Array<

In [31]:
pdf_text = "公務員懲戒委員會裁定\n" \
       "102年度清字第11347號\n" \
       "移送機關　內政部　設臺北市○○區○○路○號\n" \
       "代表人　徐國勇　住同上\n" \
       "被付懲戒人　陳仁維　臺東縣警察局警務員\n" \
       "上列被付懲戒人因違法失職案件，經內政部移送審理，本會裁定\n"\
       "如下：\n"\
       "主文\n"\
       "本件停止審議程序之議決撤銷。\n"
print(len(pdf_text))


131


In [33]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    document_extraction_results = chain_part1.predict_and_parse(text=pdf_text)
    print(f"Output: {document_extraction_results.get('data')}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")


Output: {'judgment_info_part1': [{'title': '公務品懲委員會裁定'}, {'id': '102年度清字第11347號'}, {'appeal': {'representative': ['徐國勱'], 'agent': ['徐國勱']}}, {'appealed': {'appellee': ['陳仁維'], 'agent': ['臺東縣警察局警務員']}}, {'foreword': '上列被付懲房回人因運法失聊案件事，經內政部移送审理，本會裁定如下：'}, {'main_content': '本件停止審議程序之議決撤銷。'}]}
Total Tokens: 3506
Prompt Tokens: 3134
Completion Tokens: 372
Successful Requests: 1
Total Cost (USD): $0.007012
